We can get the data for game franchises from Giantbomb. My api key is f2577eac7f52b72170fc1e8c5f0cebb778f38155

In [ ]:
import requests
import json
import pandas as pd
import re
from pathlib import Path
import time

## Acquiring and cleaning franchise IDs

Create the directory to store the raw info

In [ ]:
franchises_dir = Path('raw_json\\game_franchises\\franchises')
franchises_dir.mkdir(parents=True, exist_ok=True)

Get all the data from the franchise ids and save as json files

In [ ]:
user_agent = "Joshdharris"
headers = {'User-Agent': user_agent}

for x in range(0, 5550, 100):
    filename = "page{}-{}.json".format(x, x+99)
    if not Path(franchises_dir, filename).is_file():
        url = "https://www.giantbomb.com/api/franchises/?api_key=f2577eac7f52b72170fc1e8c5f0cebb778f38155&format=json&field_list=guid,name,description&offset={}".format(x)
        response = json.loads((requests.get(url, headers=headers)).content)
        with open(Path(franchises_dir, filename), "w") as fileOut:
            json.dump(response, fileOut)

This function will be used multiple times for the raw json files. Here we create an empty dataframe, then open all the json files in a specified directory and add them to the dataframe, before returning the dataframe

In [202]:
def create_dataframe(desired_name, raw_directory):
    #Create empty dataframe
    desired_name = pd.DataFrame(data=[], columns=None)

    #This line creates a list of all the json files in the directory
    files = list(Path(raw_directory).glob('*.json'))
    
    #Iterate through all json files in directory and add to dataframe
    for file in files:
        with open(file) as filein:
            desired_name = pd.concat([desired_name, pd.read_json(filein)])
    return desired_name

Collate all the downloaded files into a single dataframe

In [222]:
franchise_ids = create_dataframe("franchise_ids", franchises_dir)
franchise_ids

,error,limit,offset,number_of_page_results,number_of_total_results,status_code,results,version
0,OK,100,0,100,5524,1,{'description': '<h2>Navigation</h2><h3>Sub-Fr...,1
1,OK,100,0,100,5524,1,{'description': '<h2>Overview</h2><figure data...,1
2,OK,100,0,100,5524,1,{'description': '<h2>Overview</h2><p>Metroid i...,1
3,OK,100,0,100,5524,1,{'description': '<h2>Overview</h2><p>Developed...,1
4,OK,100,0,100,5524,1,{'description': '<h2>Overview</h2><p>The Metal...,1
...,...,...,...,...,...,...,...,...
95,OK,100,900,100,5524,1,{'description': '<p>Imperium Galactica is semi...,1
96,OK,100,900,100,5524,1,"{'description': None, 'guid': '3025-1030', 'na...",1
97,OK,100,900,100,5524,1,"{'description': None, 'guid': '3025-1031', 'na...",1
98,OK,100,900,100,5524,1,"{'description': None, 'guid': '3025-1032', 'na...",1


Remove unnecessary rows

Again, create function as it's used multiple times

In [220]:
def drop_unnecessary_columns(dataframe):
    dataframe = dataframe.drop(["error","limit", "offset", "number_of_page_results", "number_of_total_results", "status_code", "version"], axis = 1)
    return dataframe

In [224]:
franchise_ids = drop_unnecessary_columns(franchise_ids)
franchise_ids

,results
0,{'description': '<h2>Navigation</h2><h3>Sub-Fr...
1,{'description': '<h2>Overview</h2><figure data...
2,{'description': '<h2>Overview</h2><p>Metroid i...
3,{'description': '<h2>Overview</h2><p>Developed...
4,{'description': '<h2>Overview</h2><p>The Metal...
...,...
95,{'description': '<p>Imperium Galactica is semi...
96,"{'description': None, 'guid': '3025-1030', 'na..."
97,"{'description': None, 'guid': '3025-1031', 'na..."
98,"{'description': None, 'guid': '3025-1032', 'na..."


Extract information from results

In [225]:
franchise_ids["guid"] = [d.get('guid') for d in franchise_ids.results]
franchise_ids["name"] = [d.get('name') for d in franchise_ids.results]
franchise_ids["description"] = [d.get('description') for d in franchise_ids.results]

Find missing nubers in franchise series

In [ ]:
expected_numbers = set('3025-{}'.format(i) for i in range(1, 5699))

# create a set of actual numbers in the column
actual_numbers = set(franchise_ids['guid'])

# find the missing numbers by subtracting the actual set from the expected set
missing_numbers = expected_numbers - actual_numbers
missing_numbers

Get list of all franchise ids.

In [ ]:
franchise_id_list = franchise_ids["guid"].tolist()

## Acquiring game information for each franchise

Create directory for game franchises

In [ ]:
games_dir = Path('raw_json\\game_franchises\\games')
games_dir.mkdir(parents=True, exist_ok=True)

Download json files for each franchise

In [214]:
for id in franchise_id_list:
    filename = "{}.json".format(id)
    if not Path(games_dir, filename).is_file():
        url = "https://www.giantbomb.com/api/franchise/{}/?api_key=f2577eac7f52b72170fc1e8c5f0cebb778f38155&format=json&field_list=games".format(id)
        response = json.loads((requests.get(url, headers=headers)).content)
        with open(Path(games_dir, filename), "w") as fileOut:
            json.dump(response, fileOut)
        time.sleep(18)

Create a games dataframe from all the games in the directory

In [215]:
games_ids = create_dataframe("games_ids", games_dir)
games_ids

,error,limit,offset,number_of_page_results,number_of_total_results,status_code,results,version
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1
games,OK,1,0,1,1,1,[{'api_detail_url': 'https://www.giantbomb.com...,1


In [221]:
games_ids = drop_unnecessary_columns(games_ids)
games_ids

,results
games,[{'api_detail_url': 'https://www.giantbomb.com...
games,[{'api_detail_url': 'https://www.giantbomb.com...
games,[{'api_detail_url': 'https://www.giantbomb.com...
games,[{'api_detail_url': 'https://www.giantbomb.com...
games,[{'api_detail_url': 'https://www.giantbomb.com...


# UNTESTED CODE FROM THIS POINT ON - WORK IN PROGRESS

In [ ]:
game_database = pd.DataFrame(data=[], columns=None)

tempDF = pd.DataFrame({'guid': x, 'games': pd.read_json(r.text)["results"]})
game_database = pd.concat([game_database, tempDF])

In [ ]:
merged_df = pd.merge(noidea, game_database, on='guid', how='left')


In [ ]:
merged_df["games"].iloc[0]

In [ ]:
game_database

We have 23 franchises which contain sub-franchises. Let's extract the sub-franchises. First, look at an example of the description

In [ ]:
subfranchises = franchise_ids[franchise_ids["description"].str.contains('Sub-Franchises', flags=re.IGNORECASE, na=False, regex=True)]

In [ ]:
subfranchises["description"].iloc[3]

In [ ]:
temp = subfranchises["description"].iloc[0]
pattern = r'<a[^>]*data-ref-id="3025-[^"]*"[^>]*>[^<]*</a>'
matches = re.findall(pattern, temp)
print(matches)